# Question 2: Resampling and Frequency Conversion

This question focuses on resampling operations and frequency conversion using ICU monitoring data (hourly) and patient vital signs data (daily).

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('default')
sns.set_style('whitegrid')

# Create output directory
os.makedirs('output', exist_ok=True)

## Part 2.1: Load and Prepare Data

**Note:** These datasets have realistic characteristics:
- **ICU Monitoring**: 75 patients with variable stay lengths (2-30 days). Not all patients are present for the entire 6-month period - patients are admitted and discharged at different times.
- **Patient Vitals**: Already contains some missing visits (~5% missing data). This is realistic and will be useful for practicing missing data handling.

In [2]:
# Load ICU monitoring data (hourly)
icu_monitoring = pd.read_csv('data/icu_monitoring.csv')

# Load patient vitals data (daily) - for comparison
patient_vitals = pd.read_csv('data/patient_vitals.csv')

print("ICU monitoring shape:", icu_monitoring.shape)
print("Patient vitals shape:", patient_vitals.shape)

# Convert datetime columns and set as index
icu_monitoring['datetime'] = pd.to_datetime(icu_monitoring['datetime'])
icu_monitoring = icu_monitoring.set_index('datetime')

patient_vitals['date'] = pd.to_datetime(patient_vitals['date'])
patient_vitals = patient_vitals.set_index('date')

print("\nICU monitoring sample:")
print(icu_monitoring.head())
print("\nPatient vitals sample:")
print(patient_vitals.head())

# Check data characteristics
print(f"\nICU patients: {icu_monitoring['patient_id'].nunique()}")
print(f"ICU date range: {icu_monitoring.index.min()} to {icu_monitoring.index.max()}")
print(f"\nPatient vitals patients: {patient_vitals['patient_id'].nunique()}")
print(f"Patient vitals date range: {patient_vitals.index.min()} to {patient_vitals.index.max()}")

ICU monitoring shape: (86400, 7)
Patient vitals shape: (18250, 7)

ICU monitoring sample:
                    patient_id  heart_rate  blood_pressure_systolic  \
datetime                                                              
2023-01-01 00:00:00     ICU001   82.000000                      126   
2023-01-01 01:00:00     ICU001   98.294095                      128   
2023-01-01 02:00:00     ICU001  103.500000                      129   
2023-01-01 03:00:00     ICU001   91.535534                      136   
2023-01-01 04:00:00     ICU001   87.330127                      129   

                     blood_pressure_diastolic  oxygen_saturation  temperature  
datetime                                                                       
2023-01-01 00:00:00                        65                 96    98.783988  
2023-01-01 01:00:00                        67                 95    99.186212  
2023-01-01 02:00:00                        68                 94    98.800638  
2023-01-01 0

## Part 2.2: Time Series Selection

**⚠️ WARNING: Sort Index Before Date Selection!**
Since multiple patients share the same date, the `patient_vitals` index is non-monotonic (not strictly increasing). **You MUST sort the index first** before using `.loc` with date ranges:

In [3]:
patient_vitals = patient_vitals.sort_index()

Without sorting, pandas cannot reliably handle date range selections and may return unexpected results or errors.

**TODO: Perform time series indexing and selection**

In [4]:
# 1. Select data by specific dates
# ==============================

# January 1, 2023
january_first = patient_vitals.loc["2023-01-01"]

print("January 1, 2023 data:")
display(january_first)
print(f"Records on Jan 1: {len(january_first)} (some patients may start later)")


# ==============================
# 2. Select data by date ranges
# ==============================

# Entire January 2023
january_data = patient_vitals.loc["2023-01-01" : "2023-01-31"]

print("January 2023 shape:", january_data.shape)


# ==============================
# 3. Select data by time periods
# ==============================

# First quarter
first_quarter = patient_vitals.loc["2023-01-01" : "2023-03-31"]

# Entire 2023
entire_year = patient_vitals.loc["2023"]  # shorthand for 2023-01-01 → 2023-12-31


# ==============================
# 4. Select first and last week
# ==============================

first_week = patient_vitals.loc[: patient_vitals.index.min() + pd.Timedelta(days=6)]
last_week = patient_vitals.loc[patient_vitals.index.max() - pd.Timedelta(days=6) :]


# ==============================
# 5. Using truncate()
# ==============================

# Data after June 1, 2023
data_after_june = patient_vitals.truncate(before="2023-06-01")

# Data before September 1, 2023
data_before_september = patient_vitals.truncate(after="2023-08-31")


# ==============================
# 6. Summary analysis
# ==============================

print(f"\nFirst quarter average temperature: {first_quarter['temperature'].mean():.2f}°F")
print(f"After June average temperature: {data_after_june['temperature'].mean():.2f}°F")
print(f"First week average temperature: {first_week['temperature'].mean():.2f}°F")
print(f"Last week average temperature: {last_week['temperature'].mean():.2f}°F")


# ==============================
# 7. ICU data — time-of-day filtering
# ==============================

# Business hours 9 AM to 5 PM
business_hours = icu_monitoring.between_time("09:00", "17:00")

print("\nBusiness hours data shape:", business_hours.shape)

# Noon-only readings
noon_data = icu_monitoring.at_time("12:00")


# ==============================
# 8. Compare business vs all hours
# ==============================

all_hours_avg = icu_monitoring.select_dtypes(include=[np.number]).mean()
business_hours_avg = business_hours.select_dtypes(include=[np.number]).mean()

print(f"\nAverage heart rate - All hours: {all_hours_avg['heart_rate']:.1f} bpm")
print(f"Average heart rate - Business hours: {business_hours_avg['heart_rate']:.1f} bpm")

print(f"Average temperature - All hours: {all_hours_avg['temperature']:.1f}°F")
print(f"Average temperature - Business hours: {business_hours_avg['temperature']:.1f}°F")

January 1, 2023 data:


,patient_id,temperature,heart_rate,blood_pressure_systolic,blood_pressure_diastolic,weight
date,,,,,,
2023-01-01,P0001,98.389672,71,119,84,68.996865
2023-01-01,P0024,97.552103,71,111,70,49.386068
2023-01-01,P0025,98.806201,83,118,65,84.096164
2023-01-01,P0047,98.943464,63,110,76,54.318578
2023-01-01,P0026,98.758551,75,114,75,49.830300
2023-01-01,P0027,98.462467,90,119,74,72.943755
2023-01-01,P0028,99.632166,71,117,86,104.977255
2023-01-01,P0029,98.881121,68,121,71,59.883550
2023-01-01,P0030,98.403614,73,122,79,60.754351


Records on Jan 1: 50 (some patients may start later)
January 2023 shape: (1550, 6)

First quarter average temperature: 98.97°F
After June average temperature: 98.41°F
First week average temperature: 98.68°F
Last week average temperature: 98.65°F

Business hours data shape: (32400, 6)

Average heart rate - All hours: 81.7 bpm
Average heart rate - Business hours: 80.7 bpm
Average temperature - All hours: 98.5°F
Average temperature - Business hours: 98.5°F


## Part 2.3: Resampling Operations

**TODO: Perform resampling and frequency conversion**

**Important Note:** When resampling DataFrames that contain non-numeric columns (like `patient_id`), you'll get an error if you try to aggregate them with numeric functions like `mean()`. Use `df.select_dtypes(include=[np.number])` to select only numeric columns before resampling, or specify which columns to aggregate in `.agg()`.

In [5]:
# 1. ICU hourly → daily


# Select numeric columns only
numeric_cols_icu = icu_monitoring.select_dtypes(include=[np.number]).columns

icu_daily = icu_monitoring[numeric_cols_icu].resample("D").mean()
print("ICU daily shape:", icu_daily.shape)


# ==============================
# 2. Patient vitals daily → weekly
# ==============================

numeric_cols_pv = patient_vitals.select_dtypes(include=[np.number]).columns

patient_vitals_weekly = patient_vitals[numeric_cols_pv].resample("W").mean()
print("Weekly resampled shape:", patient_vitals_weekly.shape)


# ==============================
# 3. Patient vitals daily → monthly
# ==============================

patient_vitals_monthly = patient_vitals[numeric_cols_pv].resample("ME").mean()
print("Monthly resampled shape:", patient_vitals_monthly.shape)


# ==============================
# 4. Different aggregation functions
# ==============================

icu_daily_stats = icu_monitoring[numeric_cols_icu].resample("D").agg({
    "heart_rate": ["mean", "max", "min"],
    "temperature": ["mean", "max", "min"],
    "oxygen_saturation": "mean"
})

icu_daily_stats.head()


# ==============================
# 5. Upsampling → creates missing values
# ==============================

# Upsample monthly → daily (creates NaNs)
monthly_to_daily = patient_vitals_monthly.resample("D").asfreq()

print("Missing values after upsampling:")
print(monthly_to_daily.isna().sum())


# ==============================
# 6. Compare daily vs weekly vs monthly
# ==============================

# Step 1 — aggregate patient_vitals to daily level (mean across patients)
patient_vitals_reset = patient_vitals[numeric_cols_pv].reset_index()
patient_vitals_daily_agg = patient_vitals_reset.groupby("date").mean()

# Ensure datetime is index
patient_vitals_daily_agg.index = pd.to_datetime(patient_vitals_daily_agg.index)

# Step 2 — build summary comparison table
resampling_comparison = pd.DataFrame({
    "frequency": ["daily", "weekly", "monthly"],
    "date_range": [
        f"{patient_vitals_daily_agg.index.min().date()} → {patient_vitals_daily_agg.index.max().date()}",
        f"{patient_vitals_weekly.index.min().date()} → {patient_vitals_weekly.index.max().date()}",
        f"{patient_vitals_monthly.index.min().date()} → {patient_vitals_monthly.index.max().date()}"
    ],
    "row_count": [
        len(patient_vitals_daily_agg),
        len(patient_vitals_weekly),
        len(patient_vitals_monthly)
    ],
    "mean_temperature": [
        patient_vitals_daily_agg["temperature"].mean(),
        patient_vitals_weekly["temperature"].mean(),
        patient_vitals_monthly["temperature"].mean()
    ],
    "std_temperature": [
        patient_vitals_daily_agg["temperature"].std(),
        patient_vitals_weekly["temperature"].std(),
        patient_vitals_monthly["temperature"].std()
    ]
})

print("\nResampling comparison:")
display(resampling_comparison)


# ==============================
# 7. Save output
# ==============================

resampling_comparison.to_csv("output/q2_resampling_analysis.csv", index=False)
print("Saved: output/q2_resampling_analysis.csv")


ICU daily shape: (180, 5)
Weekly resampled shape: (53, 5)
Monthly resampled shape: (12, 5)
Missing values after upsampling:
temperature                 323
heart_rate                  323
blood_pressure_systolic     323
blood_pressure_diastolic    323
weight                      323
dtype: int64

Resampling comparison:


,frequency,date_range,row_count,mean_temperature,std_temperature
0,daily,2023-01-01 → 2023-12-31,365,98.660538,0.360322
1,weekly,2023-01-01 → 2023-12-31,53,98.660657,0.358014
2,monthly,2023-01-31 → 2023-12-31,12,98.662794,0.369503


Saved: output/q2_resampling_analysis.csv


## Part 2.4: Missing Data Handling

**💡 TIP: High Percentage of Missing Data is Expected!**
When upsampling from monthly to daily frequency, you'll create approximately 96% missing data (only 12 month-end dates have values out of 365 days). This is normal and expected for upsampling - don't be alarmed!

**Approach:** Create missing values by upsampling monthly data to daily frequency. This creates a clear, structured pattern of missing data that's ideal for practicing imputation methods.

**TODO: Handle missing data in time series**

In [11]:
monthly_temp = patient_vitals_monthly["temperature"]

# Upsample to daily frequency → creates missing values (only month-end dates have data)
ts_with_missing = monthly_temp.resample("D").asfreq()

print("Missing value count:", ts_with_missing.isna().sum())
print("Missing value percentage:", ts_with_missing.isna().sum() / len(ts_with_missing) * 100)

# Forward fill (propagate last known monthly value forward)
ts_ffill = ts_with_missing.ffill()

# Backward fill (propagate next known monthly value backward)
ts_bfill = ts_with_missing.bfill()

print("After forward fill — missing values:", ts_ffill.isna().sum())
print("After backward fill — missing values:", ts_bfill.isna().sum())

# TODO: Use interpolation methods
# Generic interpolate() – defaults to linear for numeric series
ts_interpolated = ts_with_missing.interpolate()

# Explicit linear interpolation
ts_interpolated_linear = ts_with_missing.interpolate(method="linear")

# Time-based interpolation (uses actual date spacing)
ts_interpolated_time = ts_with_missing.interpolate(method="time")

print("Missing after generic interpolate():", ts_interpolated.isna().sum())
print("Missing after linear interpolation:", ts_interpolated_linear.isna().sum())
print("Missing after time interpolation:", ts_interpolated_time.isna().sum())

# TODO: Use rolling mean for imputation
ts_filled = ts_with_missing.ffill()
rolling_mean = ts_filled.rolling(window=7, min_periods=1).mean()
ts_rolling_imputed = ts_with_missing.fillna(rolling_mean)
print("Missing after rolling mean imputation:", ts_rolling_imputed.isna().sum())


missing_by_month = ts_with_missing.groupby(ts_with_missing.index.month).apply(lambda x: x.isna().sum())
missing_by_day = ts_with_missing.groupby(ts_with_missing.index.dayofweek).apply(lambda x: x.isna().sum())

missing_patterns = f"Missing by month:\n{missing_by_month}\n\nMissing by day of week:\n{missing_by_day}"

# 2. Create full 300+ word report
missing_data_report = f"""
MISSING DATA REPORT — TIME SERIES (Q2.4)

1. Missing Value Summary
The upsampled daily time series created from monthly temperature data contained a very high rate of missing values. 
After resampling from month-end data to daily frequency using `.resample('D').asfreq()`, the resulting series had 
{ts_with_missing.isna().sum()} missing values out of {len(ts_with_missing)} total observations, which is approximately 
{ts_with_missing.isna().sum() / len(ts_with_missing) * 100:.2f}% missing. This occurs because only 12 monthly values exist, 
and all non–month-end dates become NaN.

2. Missing Data Patterns
The missing data pattern is extremely structured and deterministic: missing values occur on every day except the last 
day of each month. The missing-by-month summary shows that longer months (e.g., January, March, July, August) contain 
more missing values simply because they have more non–month-end days. Similarly, the missing-by-weekday pattern shows 
that missing values are spread evenly across weekdays, reflecting the fact that missingness is generated by calendar 
structure rather than patient behavior or data collection issues. This pattern is typical of upsampling rather than 
real-world data gaps.

Missing by month:
{missing_by_month}

Missing by day of week:
{missing_by_day}

3. Imputation Method
I used several methods to explore different approaches to filling missing values: forward fill, backward fill, linear 
interpolation, time-based interpolation, and rolling-mean imputation. Of these, the rolling-mean imputation method 
produced the smoothest and most clinically meaningful trajectory, although it required an initial forward fill step to 
avoid propagating NaNs in long gaps.

4. Rationale
Forward fill is simple and preserves the last observed value, but it creates flat segments that may not reflect real 
physiology. Backward fill has the same issue. Linear interpolation works well for evenly spaced monthly data, but it may 
smooth too aggressively. Time-based interpolation is preferred in medical time series because it respects uneven temporal 
spacing. Rolling-mean imputation provides local smoothing and reduces noise, making it appropriate for clinical vital 
signs that tend to evolve gradually.

5. Pros and Cons
Advantages: Rolling mean reduces noise, avoids abrupt jumps, and produces realistic trends. Interpolation methods 
maintain continuity and avoid sudden step changes introduced by ffill/bfill.  
Limitations: Rolling windows require assumptions about local smoothness, and forward fill must be used first to anchor 
the window. Interpolation may produce unrealistic values if underlying dynamics are non-linear.

6. Example
Before imputation, January had NaNs on every day except 2023-01-31. After applying rolling-mean imputation, all missing 
values were filled, resulting in a continuous daily temperature series suitable for downstream analysis.
"""

# 3. Save report
with open('output/q2_missing_data_report.txt', 'w') as f:
    f.write(missing_data_report)
    f.write("\n\n--- Missing Patterns ---\n")
    f.write(missing_patterns)

print("Saved: output/q2_missing_data_report.txt")

Missing value count: 323
Missing value percentage: 96.41791044776119
After forward fill — missing values: 0
After backward fill — missing values: 0
Missing after generic interpolate(): 0
Missing after linear interpolation: 0
Missing after time interpolation: 0
Missing after rolling mean imputation: 0
Saved: output/q2_missing_data_report.txt


## Submission Checklist

Before moving to Question 3, verify you've created:

- [ ] `output/q2_resampling_analysis.csv` - resampling analysis results
- [ ] `output/q2_missing_data_report.txt` - missing data handling report
